In [1]:
from win32com import client
from comtypes.client import CreateObject
import win32com.client
import pandas as pd
from docx import Document
from jinja2 import Environment, FileSystemLoader
import xml.etree.ElementTree as ET

In [2]:

model_path = "C:/Users/AngeloNiforatos/OneDrive - ResilienX Inc/Documents/Resilienx_Repos/model.qea"
# Create EA Repository object
# ea_repository = win32com.client.Dispatch("EA.Repository")
eaApp = win32com.client.Dispatch("EA.App")
ea_repository = eaApp.Repository

# Connect to a model (replace ModelPath with your actual model file path)
# ea_repository.OpenFile(model_path)
ea_repository

<COMObject <unknown>>

In [3]:
active_diagram = ea_repository.GetCurrentDiagram()
diagram_id = active_diagram.DiagramID; name = active_diagram.name
print(f"Diagram ID: {diagram_id}\nName: {name}")

Diagram ID: 1199
Name: Monitor Network Infrastructure


In [4]:
# Function to execute SQL query and return a DataFrame
def execute_sql_query(ea_repository, sql_query):

    # Execute the SQL query
    result_set = ea_repository.SQLQuery(sql_query)
    print(result_set)
    # Parse the XML data
    root = ET.fromstring(result_set)
    root = root.find("Dataset_0").find("Data")
    # Extract column names from the first row
    column_names = [element.tag for element in root[0]]

    # Extract data from XML and create a list of dictionaries
    data = [{element.tag: element.text for element in row} for row in root]

    # Convert the list of dictionaries to a Pandas DataFrame
    df = pd.DataFrame(data, columns=column_names)
    return df

# Function to generate a Word document using a Jinja2 template
def generate_document(data, template_path, output_path):
    env = Environment(loader=FileSystemLoader('.'))
    template = env.get_template(template_path)

    document = Document()
    
    for row in data.itertuples(index=False):
        document.add_paragraph(template.render(row))

    document.save(output_path)

In [11]:
sql_query = """SELECT
	t_object.Name as RequirementName,
	REPLACE(REPLACE(t_object.Note, '<b>', ''), '</b>', '') AS CleanedNotes,
	t_diagram.Name as DiagramName
FROM
	t_object
JOIN
	t_diagramobjects ON t_diagramobjects.Object_ID = t_object.Object_ID
JOIN
	t_diagram ON t_diagram.Diagram_ID = t_diagramobjects.Diagram_ID
WHERE
	t_object.Object_Type = "Requirement" 
	/* Specify the Diagram GUI ID */
	AND t_diagram.ea_guid = "{555FAF93-3E41-4c95-B76D-43A809F83978}";"""
sql_query_comments = f"""SELECT
    t_diagram.Name AS DiagramName,
    t_object.Name AS ElementName,
    t_object.Note AS Comment,
	t_object.Object_Type as Type
FROM
    t_diagramobjects
JOIN
    t_diagram ON t_diagram.Diagram_ID = t_diagramobjects.Diagram_ID
JOIN
    t_object ON t_object.Object_ID = t_diagramobjects.Object_ID

WHERE
	(t_object.Object_Type = 'Note' OR t_object.Stereotype = 'Note') AND
    t_diagram.Diagram_ID = {diagram_id};"""
sql_query_comments

"SELECT\n    t_diagram.Name AS DiagramName,\n    t_object.Name AS ElementName,\n    t_object.Note AS Comment,\n\tt_object.Object_Type as Type\nFROM\n    t_diagramobjects\nJOIN\n    t_diagram ON t_diagram.Diagram_ID = t_diagramobjects.Diagram_ID\nJOIN\n    t_object ON t_object.Object_ID = t_diagramobjects.Object_ID\n\nWHERE\n\t(t_object.Object_Type = 'Note' OR t_object.Stereotype = 'Note') AND\n    t_diagram.Diagram_ID = 1199;"

In [12]:
df = execute_sql_query(ea_repository, sql_query_comments)
df

<?xml version="1.0" encoding="UTF-16" standalone="no" ?>
<EADATA version="1.0" exporter="Enterprise Architect"><Dataset_0><Data><Row><DiagramName>Monitor Network Infrastructure</DiagramName><ElementName xmlns:dt="urn:schemas-microsoft-com:datatypes" dt:dt="bin.base64"/><Comment>MS: Some simplifications recommended:&#xD;
I believe FAA126 requires the system to trigger a network fault's mitigation when that component's preconfigured fault is annotated as a network monitoring kind of fault. &#xD;
FAA126 can derive from IASMS400, and so can FAA127. Additionally, 126 should derive from 051. 127 should derive from 045.&#xD;
You may consider a requirement deriving from all fault type requirements (current 045 and 051): The system shall generate a manual mitigation upon creating a Network Monitoring Fault.&#xD;
This would address FAA126 and FAA127, but if you want to keep those broken out for the types of faults (and have the ability to specialize the functionality of each), keep as-is.&#xD;
S

,DiagramName,ElementName,Comment,Type
0,Monitor Network Infrastructure,None,MS: Some simplifications recommended:\r\nI bel...,Note
1,Monitor Network Infrastructure,None,MS: This requirement should be a bit more prec...,Note
2,Monitor Network Infrastructure,None,"BT: Question for Greg, do all components need ...",Note
3,Monitor Network Infrastructure,None,BT: also derived from FAA122\r\nAN: Agree!,Note
4,Monitor Network Infrastructure,None,BT: Two of the same titles? Should we derive o...,Note
5,Monitor Network Infrastructure,None,"BT: It's not just ping latency, right? that's ...",Note
6,Monitor Network Infrastructure,None,"BT: I see what you are trying to do here, but ...",Note
7,Monitor Network Infrastructure,None,BT: not sure we can do liveliness of network c...,Note


In [20]:
# Replace 'output_file.txt' with your desired output file path
output_file_path = 'output_file.html'

# Extract the contents of the 'FormattedTextColumn'
formatted_text_contents = df['Comment'].tolist()

# Write the contents to a rich-text HTML file
with open(output_file_path, 'w', encoding='utf-8') as html_file:
    # Write the HTML header
    html_file.write('<html>\n<head></head>\n<body>\n')

    # Write each formatted text content
    for content in formatted_text_contents:
        html_file.write(f'{content}<br><br>')

    # Write the HTML footer
    html_file.write('</body>\n</html>')